<p style="align: center;"><img src="https://static.tildacdn.com/tild6636-3531-4239-b465-376364646465/Deep_Learning_School.png" width="400"></p>

# Домашнее задание. Обучение языковой модели с помощью LSTM (10 баллов)

Э
В этом задании Вам предстоит обучить языковую модель с помощью рекуррентной нейронной сети. В отличие от семинарского занятия, Вам необходимо будет работать с отдельными словами, а не буквами.


Установим модуль ```datasets```, чтобы нам проще было работать с данными.

In [1]:
!pip install -U datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 17.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 13.4 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
  Attempting uninstall: datasets
    Found existing installation: datasets 2.14.4
    Uninstalling datasets-2.14.4:
      Successfully uninstalled datasets-2.14.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2025.3.0 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; pl

Импорт необходимых библиотек

In [2]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

import numpy as np
import matplotlib.pyplot as plt

from tqdm.auto import tqdm
from datasets import load_dataset
from nltk.tokenize import sent_tokenize, word_tokenize
from sklearn.model_selection import train_test_split
import nltk

from collections import Counter
from typing import List

import seaborn
seaborn.set(palette='summer')

In [3]:
nltk.download('punkt')
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [4]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

## Подготовка данных

Воспользуемся датасетом imdb. В нем хранятся отзывы о фильмах с сайта imdb. Загрузим данные с помощью функции ```load_dataset```

In [5]:
# Загрузим датасет
dataset = load_dataset('imdb')

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/21.0M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/20.5M [00:00<?, ?B/s]

unsupervised-00000-of-00001.parquet:   0%|          | 0.00/42.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

### Препроцессинг данных и создание словаря (1 балл)

Далее вам необходмо самостоятельно произвести препроцессинг данных и получить словарь или же просто ```set``` строк. Что необходимо сделать:

1. Разделить отдельные тренировочные примеры на отдельные предложения с помощью функции ```sent_tokenize``` из бибилиотеки ```nltk```. Каждое отдельное предложение будет одним тренировочным примером.
2. Оставить только те предложения, в которых меньше ```word_threshold``` слов.
3. Посчитать частоту вхождения каждого слова в оставшихся предложениях. Для деления предлоения на отдельные слова удобно использовать функцию ```word_tokenize```.
4. Создать объект ```vocab``` класса ```set```, положить в него служебные токены '\<unk\>', '\<bos\>', '\<eos\>', '\<pad\>' и vocab_size самых частовстречающихся слов.   

In [76]:
sentences = []
word_threshold = 80

# Получить отдельные предложения и поместить их в sentences
for sentence in tqdm(dataset['train']['text']):
  sentences.extend(
      [x.lower() for x in sent_tokenize(sentence, language='english') if len(x) < word_threshold]
      )

  0%|          | 0/25000 [00:00<?, ?it/s]

In [77]:
print("Всего предложений:", len(sentences))

Всего предложений: 107275


In [78]:
print(sentences[3])

nowhere, because they don't exist.


Посчитаем для каждого слова его встречаемость.

In [79]:
words = Counter()

# Расчет встречаемости слов
for sentence in tqdm(sentences):
  word_tokens = word_tokenize(sentence)
  for word in word_tokens:
    words[word] += 1

  0%|          | 0/107275 [00:00<?, ?it/s]

In [80]:
print("Всего уникальных слов:", len(words))

Всего уникальных слов: 36126


In [81]:
print(words)

Counter({'.': 86183, 'the': 45447, ',': 33038, 'a': 25472, 'is': 23231, 'it': 22813, 'i': 22223, 'and': 19916, 'this': 19135, 'to': 17848, 'of': 17760, '!': 13728, 'in': 12031, 'was': 11682, "'s": 11195, 'movie': 10975, 'that': 10107, '?': 9040, "n't": 8009, 'you': 7340, '<': 7276, '/': 7242, '>': 7229, 'br': 7203, 'for': 7030, 'film': 6983, 'but': 6819, 'not': 6817, 'as': 5536, 'he': 5381, '``': 5293, 'are': 5131, "''": 5053, 'one': 4933, 'have': 4932, 'on': 4911, 'with': 4869, 'be': 4628, 'all': 4576, 'do': 4445, 'they': 4073, 'just': 3951, 'so': 3869, 'what': 3856, 'like': 3639, 'at': 3633, 'good': 3631, 'there': 3630, ')': 3587, 'his': 3154, 'no': 3152, '(': 3058, 'very': 3043, 'my': 3029, 'an': 2930, 'if': 2883, 'out': 2826, 'about': 2769, 'has': 2754, 'really': 2730, 'she': 2693, 'did': 2669, 'bad': 2628, 'me': 2571, 'or': 2498, 'see': 2429, 'can': 2394, 'from': 2334, 'even': 2320, 'by': 2272, 'her': 2235, 'time': 2204, 'more': 2174, 'great': 2161, 'does': 2150, 'would': 2150, 'w

Добавим в словарь ```vocab_size``` самых встречающихся слов.

In [82]:
vocab = set()
vocab_size = 20_000

temp = words.most_common(vocab_size)
print(temp)


for word, _ in words.most_common(vocab_size):
        vocab.add(word)

# Служебные токены
vocab.add('<unk>')
vocab.add('<bos>')
vocab.add('<eos>')
vocab.add('<pad>')

[('.', 86183), ('the', 45447), (',', 33038), ('a', 25472), ('is', 23231), ('it', 22813), ('i', 22223), ('and', 19916), ('this', 19135), ('to', 17848), ('of', 17760), ('!', 13728), ('in', 12031), ('was', 11682), ("'s", 11195), ('movie', 10975), ('that', 10107), ('?', 9040), ("n't", 8009), ('you', 7340), ('<', 7276), ('/', 7242), ('>', 7229), ('br', 7203), ('for', 7030), ('film', 6983), ('but', 6819), ('not', 6817), ('as', 5536), ('he', 5381), ('``', 5293), ('are', 5131), ("''", 5053), ('one', 4933), ('have', 4932), ('on', 4911), ('with', 4869), ('be', 4628), ('all', 4576), ('do', 4445), ('they', 4073), ('just', 3951), ('so', 3869), ('what', 3856), ('like', 3639), ('at', 3633), ('good', 3631), ('there', 3630), (')', 3587), ('his', 3154), ('no', 3152), ('(', 3058), ('very', 3043), ('my', 3029), ('an', 2930), ('if', 2883), ('out', 2826), ('about', 2769), ('has', 2754), ('really', 2730), ('she', 2693), ('did', 2669), ('bad', 2628), ('me', 2571), ('or', 2498), ('see', 2429), ('can', 2394), (

In [83]:
assert '<unk>' in vocab
assert '<bos>' in vocab
assert '<eos>' in vocab
assert '<pad>' in vocab
assert len(vocab) == vocab_size + 4

In [84]:
print("Всего слов в словаре:", len(vocab))

Всего слов в словаре: 20004


In [85]:
print(vocab)

{'minutes', 'stein', 'dalton', 'jody', 'restaurant', 'leading', 'cameo', 'miscasting', 'stephanie', 'bashed', 'stir', 'wolsky', 'boroughs', 'non-stop', 'lowered', 'quenton', 'crunch', 'stink', 'superlatives', 'rolling', 'corky', 'ahista', 'brush', 'deep', 'offenders', 'bring', 'cuter', '21st', 'textbooks', 'appearances', 'shotguns', 'associates', 'pyramid', 'appeal', 'cheadle', 'transforming', 'sub-plots', 'richie', 'choreographed', 'europe', 'wikipedia', 'rounders', 'collaboration', 'miguel', 'now-cancelled', 'stylize', 'dankness', 'hickock', 'cunnilingus', 'withstood', 'similarly', 'blessings', 'investment', 'crockett', 'predecessor', 'ang', 'traci', 'overhead', 'scant', 're-made', 'cabinet', 'aided', 'worthwhile', 'yum', 'out.', 'taping', 'predictability', 'capture', 'understanding', 'holds', 'approaches', 'aggravating', 'sidewalk', 'genma', 'baseketball', 'bribes', 'keen', 'scumbags', 'stood', 'fairy-tale', 'kerry', 'wyoming', 'blood-soaked', 'overnight', 'ops', 'besties4lyf', 'fil

### Подготовка датасета (1 балл)

Далее, как и в семинарском занятии, подготовим датасеты и даталоадеры.

В классе ```WordDataset``` вам необходимо реализовать метод ```__getitem__```, который будет возвращать сэмпл данных по входному idx, то есть список целых чисел (индексов слов).

Внутри этого метода необходимо добавить служебные токены начала и конца последовательности, а также токенизировать соответствующее предложение с помощью ```word_tokenize``` и сопоставить ему индексы из ```word2ind```.

In [86]:
word2ind = {char: i for i, char in enumerate(vocab)}
ind2word = {i: char for char, i in word2ind.items()}

In [87]:
print(word2ind)

{'minutes': 0, 'stein': 1, 'dalton': 2, 'jody': 3, 'restaurant': 4, 'leading': 5, 'cameo': 6, 'miscasting': 7, 'stephanie': 8, 'bashed': 9, 'stir': 10, 'wolsky': 11, 'boroughs': 12, 'non-stop': 13, 'lowered': 14, 'quenton': 15, 'crunch': 16, 'stink': 17, 'superlatives': 18, 'rolling': 19, 'corky': 20, 'ahista': 21, 'brush': 22, 'deep': 23, 'offenders': 24, 'bring': 25, 'cuter': 26, '21st': 27, 'textbooks': 28, 'appearances': 29, 'shotguns': 30, 'associates': 31, 'pyramid': 32, 'appeal': 33, 'cheadle': 34, 'transforming': 35, 'sub-plots': 36, 'richie': 37, 'choreographed': 38, 'europe': 39, 'wikipedia': 40, 'rounders': 41, 'collaboration': 42, 'miguel': 43, 'now-cancelled': 44, 'stylize': 45, 'dankness': 46, 'hickock': 47, 'cunnilingus': 48, 'withstood': 49, 'similarly': 50, 'blessings': 51, 'investment': 52, 'crockett': 53, 'predecessor': 54, 'ang': 55, 'traci': 56, 'overhead': 57, 'scant': 58, 're-made': 59, 'cabinet': 60, 'aided': 61, 'worthwhile': 62, 'yum': 63, 'out.': 64, 'taping'

In [88]:
class WordDataset:
    def __init__(self, sentences):
        self.data = sentences
        self.unk_id = word2ind['<unk>']
        self.bos_id = word2ind['<bos>']
        self.eos_id = word2ind['<eos>']
        self.pad_id = word2ind['<pad>']

    def __getitem__(self, idx: int) -> List[int]:
        tokenized_sentence = []
        # Допишите код здесь

        #bos, text, eos
        tokenized_sentence = [self.bos_id]
        sentence = self.data[idx]
        words_tokenized = word_tokenize(sentence, language = 'english')
        tokenized_sentence += [word2ind.get(word, self.unk_id) for word in words_tokenized]
        tokenized_sentence += [self.eos_id]

        return tokenized_sentence

    def __len__(self) -> int:
        return len(self.data)

In [89]:
def collate_fn_with_padding(
    input_batch: List[List[int]], pad_id=word2ind['<pad>']) -> torch.Tensor:
    seq_lens = [len(x) for x in input_batch]
    max_seq_len = max(seq_lens)

    new_batch = []
    for sequence in input_batch:
        for _ in range(max_seq_len - len(sequence)):
            sequence.append(pad_id)
        new_batch.append(sequence)

    sequences = torch.LongTensor(new_batch).to(device)

    new_batch = {
        'input_ids': sequences[:,:-1],
        'target_ids': sequences[:,1:]
    }

    return new_batch

In [90]:
train_sentences, eval_sentences = train_test_split(sentences, test_size=0.2)
eval_sentences, test_sentences = train_test_split(sentences, test_size=0.5)

train_dataset = WordDataset(train_sentences)
eval_dataset = WordDataset(eval_sentences)
test_dataset = WordDataset(test_sentences)

batch_size = 128

train_dataloader = DataLoader(
    train_dataset, collate_fn=collate_fn_with_padding, batch_size=batch_size)

eval_dataloader = DataLoader(
    eval_dataset, collate_fn=collate_fn_with_padding, batch_size=batch_size)

test_dataloader = DataLoader(
    test_dataset, collate_fn=collate_fn_with_padding, batch_size=batch_size)

## Обучение и архитектура модели

Вам необходимо на практике проверить, что влияет на качество языковых моделей. В этом задании нужно провести серию экспериментов с различными вариантами языковых моделей и сравнить различия в конечной перплексии на тестовом множестве.

Возмоэные идеи для экспериментов:

* Различные RNN-блоки, например, LSTM или GRU. Также можно добавить сразу несколько RNN блоков друг над другом с помощью аргумента num_layers. Вам поможет официальная документация [здесь](https://pytorch.org/docs/stable/generated/torch.nn.LSTM.html)
* Различные размеры скрытого состояния. Различное количество линейных слоев после RNN-блока. Различные функции активации.
* Добавление нормализаций в виде Dropout, BatchNorm или LayerNorm
* Различные аргументы для оптимизации, например, подбор оптимального learning rate или тип алгоритма оптимизации SGD, Adam, RMSProp и другие
* Любые другие идеи и подходы

После проведения экспериментов необходимо составить таблицу результатов, в которой описан каждый эксперимент и посчитана перплексия на тестовом множестве.

Учтите, что эксперименты, которые различаются, например, только размером скрытого состояния или количеством линейных слоев считаются, как один эксперимент.

Успехов!

### Функция evaluate (1 балл)

Заполните функцию ```evaluate```

In [91]:
def evaluate(model, criterion, dataloader) -> float:
    model.eval()
    perplexity = []
    with torch.no_grad():
        for batch in dataloader:
            logits = model(batch['input_ids']).flatten(start_dim = 0, end_dim = 1)
            loss = criterion(logits, batch['target_ids'].flatten())
            perplexity.append(torch.exp(loss).item())

    perplexity = sum(perplexity) / len(perplexity)

    return perplexity

### Train loop (1 балл)

Напишите функцию для обучения модели.

In [97]:
def generate_sequence(model, starting_seq: str, max_seq_len: int = 32) -> str:
    word_tokens = word_tokenize(starting_seq)
    input_ids = [word2ind['<bos>']] + [
        word2ind.get(word, word2ind['<unk>']) for word in word_tokens]
    input_ids = torch.LongTensor(input_ids).to(device)

    model.eval()
    with torch.no_grad():
        for i in range(max_seq_len):
            next_char_distribution = model(input_ids)[-1]
            next_char = next_char_distribution.squeeze().argmax()
            input_ids = torch.cat([input_ids, next_char.unsqueeze(0)])

            if next_char.item() == word2ind['<eos>']:
                break

    words = ' '.join([ind2word[idx.item()] for idx in input_ids])

    return words

In [98]:
def train_model(model, opt, dataloader, dataloader_val, epochs = 10):
    losses = []
    perplexities = []
    perplexities_val = []
    criterion = nn.CrossEntropyLoss(ignore_index=word2ind['<pad>'])
    for epoch in range(epochs):
        epoch_losses = []
        model.train()
        for batch in tqdm(dataloader, desc=f'Training epoch {epoch}:'):
            opt.zero_grad()
            logits = model(batch['input_ids']).flatten(start_dim = 0, end_dim = 1)

            loss = criterion(logits, batch['target_ids'].flatten())
            loss.backward()
            opt.step()

            epoch_losses.append(loss.item())

        losses.append(sum(epoch_losses) / len(epoch_losses))
        perplexities.append(evaluate(model, criterion, dataloader))
        perplexities_val.append(evaluate(model, criterion, dataloader_val))
        print(generate_sequence(model, starting_seq=''))


    return losses, perplexities, perplexities_val

### Первый эксперимент (2 балла)

Определите архитектуру модели и обучите её.

In [99]:
class LanguageModel(nn.Module):
    def __init__(self, hidden_dim: int, vocab_size: int):
        super().__init__()

        # Опишите свою нейронную сеть здесь
        self.embeddings = nn.Embedding(vocab_size, hidden_dim)
        self.rnn = nn.GRU(hidden_dim, hidden_dim, num_layers = 3, batch_first=True)
        self.linear = nn.Linear(hidden_dim, hidden_dim)
        self.projection = nn.Linear(hidden_dim, vocab_size)

        self.activation = nn.Tanh()
        self.dropout = nn.Dropout(p = 0.2)

    def forward(self, input_batch: torch.Tensor) -> torch.Tensor:
        # А тут опишите forward pass модели
        emb = self.embeddings(input_batch)
        out, _ = self.rnn(emb)
        out = self.dropout(self.linear(self.activation(out)))
        out = self.projection(self.activation(out))

        return out

In [95]:
# Обучите модель здесь
model = LanguageModel(hidden_dim=64, vocab_size=len(vocab)).to(device)
opt = torch.optim.Adam(model.parameters())

In [100]:
losses, perplexities, perplexities_val = train_model(model, opt, train_dataloader, eval_dataloader, epochs = 10)

Training epoch 0::   0%|          | 0/671 [00:00<?, ?it/s]

<bos> i 'm a good movie , i have seen the movie . <eos>


Training epoch 1::   0%|          | 0/671 [00:00<?, ?it/s]

<bos> i 'm not a good movie . <eos>


Training epoch 2::   0%|          | 0/671 [00:00<?, ?it/s]

...

<bos> i 'm not a fan of the time . <eos>


Training epoch 8::   0%|          | 0/671 [00:00<?, ?it/s]

<bos> i 'm not a fan of the movie , but i 'm not a fan of the movie . <eos>


Training epoch 9::   0%|          | 0/671 [00:00<?, ?it/s]

<bos> i 'm not a fan of the time . <eos>


In [34]:
plt.plot(np.arange(len(losses)), losses)
plt.title('Losses')
plt.xlabel("epoch")
plt.show()

NameError: name 'losses' is not defined

In [35]:
plt.plot(np.arange(len(perplexities)), perplexities, label = 'train')
plt.plot(np.arange(len(perplexities_val)), perplexities_val, label = 'val')
plt.title('Perplexity')
plt.xlabel("epoch")
plt.legend()
plt.show()

NameError: name 'perplexities' is not defined

In [36]:
generate_sequence(model, starting_seq='minutes')

'<bos><unk>i<unk><unk><unk><unk><unk>.<eos>'

### Второй эксперимент (2 балла)

Попробуйте что-то поменять в модели или в пайплайне обучения, идеи для экспериментов можно подсмотреть выше.

In [ ]:
# Проведите второй эксперимент

### Отчет (2 балла)

Опишите проведенные эксперименты. Сравните перплексии полученных моделей. Предложите идеи по улучшению качества моделей.